In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
train_raw_data=pd.read_csv('../input/titanic/train.csv')
test_raw_data=pd.read_csv('../input/titanic/test.csv')

In [ ]:
trainrow=train_raw_data.shape[0]
testrow=test_raw_data.shape[0]
y_train=train_raw_data['Survived'].copy()
train_raw_data=train_raw_data.drop(['Survived'],1)

In [ ]:
combine=pd.concat([train_raw_data,test_raw_data])
combine.head()

In [ ]:
print(combine.shape)
print(train_raw_data.shape)
print(test_raw_data.shape)

In [ ]:
combine.isnull().sum()

In [ ]:
#here we can se only three features has missing data
#Emabarked lets replace it with most frequent data
combine['Embarked']=combine['Embarked'].fillna(combine['Embarked'].value_counts().index[0])

In [ ]:
combine['Cabin']=combine['Cabin'].fillna('U')
combine['Cabin'].value_counts()
combine['Cabin']=combine['Cabin'].astype(str).str[0]
combine.head()

In [ ]:
combine.loc[combine['Fare'].isnull()]

In [ ]:
#fare depend on Pclass,Sex, and age
combine['Fare']=combine['Fare'].fillna(combine.loc[(combine['Pclass']==3) & (combine['Sex']=="male") & (combine['Age']<65) & (combine['Age']>55)].dropna()['Fare'].mean())

In [ ]:
combine.head()

In [ ]:
passengerids=test_raw_data['PassengerId']
combine=combine.drop(['PassengerId','Ticket'],1)

In [ ]:
combine.head()

In [ ]:
combine['familysize']=combine['SibSp']+combine['Parch']+1
combine.head()

In [ ]:
combine['Title'] = combine.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
combine.head()

In [ ]:
combine.Title.unique()

In [ ]:
combine['Title'].value_counts()

In [ ]:
# combine['Title']=combine['Title'].map({"Mr":0,"Miss":1,"Mrs":2,"Master":3,"Dr":3,"Rev":3,"Major":3,"Col":3,"Mlle":3,"Don":3,"Jonkheer":3,"Countess":3,"Sir":0,"Capt":3,"Mme":2,"Lady":1,"Ms":0,"Dona":3})
# combine.head()

In [ ]:
combine=combine.drop(['Name'],1)
combine.head()

In [ ]:
combine=combine.drop(['SibSp','Parch'],1)
combine.head()

In [ ]:
combine['Sex']=combine['Sex'].map({'male':0,'female':1})
combine.head()

In [ ]:
print(combine.Pclass.unique())
print(combine.Sex.unique())

In [ ]:
for i in range(0,2):
    for j in range(0,3):
        print(i,j+1)
        temp_dataset=combine[(combine['Sex']==i) &  (combine['Pclass']==j+1)]['Age'].dropna()
        print(temp_dataset)
        #print(str(temp_dataset.median())+"  "+str(i)+"  "+str(j+1))
        combine.loc[(combine.Age.isnull()) & (combine.Sex==i) & (combine.Pclass==j+1),'Age']=int(temp_dataset.median())

In [ ]:
combine.isnull().sum()

In [ ]:
combine.head()

In [ ]:
combine_checkpoint=combine.copy()
combine.head()

In [ ]:
combine=combine_checkpoint.copy()
combine.head()

In [ ]:
combine['Age_Band']=pd.cut(combine['Age'],5)
combine['Age_Band'].unique()

In [ ]:
combine.loc[(combine['Age']<=16.136),'Age']=1
combine.loc[(combine['Age']>16.136) & (combine['Age']<=32.102),'Age']=2
combine.loc[(combine['Age']>32.102) & (combine['Age']<=48.068),'Age']=3
combine.loc[(combine['Age']>48.068) & (combine['Age']<=64.034),'Age']=4
combine.loc[(combine['Age']>64.034) & (combine['Age']<=80.),'Age']=5
combine['Age'].unique()

In [ ]:
combine=combine.drop(['Age_Band'],1)

In [ ]:
combine['Fare_Band']=pd.cut(combine['Fare'],3)
combine['Fare_Band'].unique()

In [ ]:
combine.loc[(combine['Fare']<=170.776),'Fare']=1
combine.loc[(combine['Fare']>170.776) & (combine['Fare']<=314.553),'Fare']=2
combine.loc[(combine['Fare']>314.553) & (combine['Fare']<=513),'Fare']=3
combine=combine.drop(['Fare_Band'],1)

In [ ]:
combine['Fare'].value_counts()

In [ ]:
combine=pd.get_dummies(columns=['Pclass','Sex','Cabin','Embarked','Title','Age','Fare'],data=combine)
combine.head()

In [ ]:
x_train=combine.iloc[:trainrow]
x_test=combine.iloc[trainrow:]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()
scaler.fit(x_train)
x_scaled_train=scaler.transform(x_train)
x_scaled_train

In [ ]:
x_scaled_test=scaler.transform(x_test)
x_scaled_test

In [ ]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(x_scaled_train,y_train)
print(reg.score(x_scaled_train,y_train))
y_pred=reg.predict(x_scaled_test)
y_pred

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_scaled_train,y_train,early_stopping_rounds=5, 
             eval_set=[(x_scaled_train, y_train)], 
             verbose=False)
print(xgb.score(x_scaled_train,y_train))
y_pred=xgb.predict(x_scaled_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=4,n_estimators=500,warm_start=True,max_depth=6,min_samples_leaf=2,max_features='sqrt')
rfc.fit(x_scaled_train,y_train)
print(rfc.score(x_scaled_train,y_train))
y_pred=rfc.predict(x_scaled_test)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors = 2)
# classifier.fit(x_scaled_train,y_train)
# print(classifier.score(x_scaled_train,y_train))
# y_pred=classifier.predict(x_scaled_test)

In [ ]:
passengerids.head()

In [ ]:
submission = pd.DataFrame({
        "PassengerId": passengerids,
        "Survived": y_pred
    })
submission

In [ ]:
submission.to_csv('submission1.csv', index=False)


<h1>Check out my other Notebooks</h1><font size='4'>
<a href="https://www.kaggle.com/vishalvanpariya/top-5-on-leaderboard" target="_blank">House Price</a><br>
<a href="https://www.kaggle.com/vishalvanpariya/data-explanation-titanic" target="_blank">Titanic EDA</a><br>
<a href="https://www.kaggle.com/vishalvanpariya/nlp-for-beginners" target="_blank">NLP</a><br><font>
<a href="https://www.kaggle.com/vishalvanpariya/basic-visualization-for-beginners" target="_blank">Basic Visualization</a><br><font>